In [1]:
import pandas as pd
import math as m
import numpy as np
import matplotlib.pyplot as plt
from time import time
from scipy import stats

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestRegressor


Bad key "“backend" on line 1 in
/Users/rfdanti/.matplotlib/matplotlibrc.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.1/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
trainset = pd.read_csv('train_merged_v0.6.csv')
trainset.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,...,Customers,DayOfWeek,Open,Promo,Sales,SchoolHoliday,StateHoliday,competition_age_days,affectedbycompetition,affectedbypromo2
0,1,2,0,1270.0,9,2008,0,1,2016,0,...,560.0,1,1,1,4652.0,1,0,2240,1,0
1,1,2,0,1270.0,9,2008,0,1,2016,0,...,657.0,1,1,1,6377.0,1,0,1778,1,0
2,1,2,0,1270.0,9,2008,0,1,2016,0,...,607.0,1,1,1,5394.0,0,0,1603,1,0
3,1,2,0,1270.0,9,2008,0,1,2016,0,...,605.0,1,1,1,5695.0,0,0,2387,1,0
4,1,2,0,1270.0,9,2008,0,1,2016,0,...,615.0,1,1,1,5325.0,0,0,2072,1,0


In [29]:
trainset["Store"][trainset["Open"]==1][trainset["DayOfWeek"]==7].unique()

array([  85,  122,  209,  259,  262,  274,  299,  310,  335,  353,  423,
        433,  453,  494,  512,  524,  530,  562,  578,  676,  682,  732,
        733,  769,  863,  867,  877,  931,  948, 1045, 1081, 1097, 1099])

In [98]:
trainset[trainset["Store"]==85].pivot_table(index=["Store","Open"],values=["Date"],
                  columns=["DayOfWeek"],aggfunc=pd.Series.nunique,fill_value=0)

Date                              
DayOfWeek     1    2    3    4    5    6    7
Store Open                                   
85    1     134  135  135  135  135  134  134

In [96]:
type1 = list([122,209,259,274,299,310,353,433,453,512,524,530,578,676,732,863,867,877,931,948,1045,1081,1099])

In [97]:
type2 = list([85,262,335,423,494,562,682,733,769,1097])

In [105]:
trainset["OpeningType"] = np.nan
trainset.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,...,DayOfWeek,Open,Promo,Sales,SchoolHoliday,StateHoliday,competition_age_days,affectedbycompetition,affectedbypromo2,OpeningType
0,1,2,0,1270.0,9,2008,0,1,2016,0,...,1,1,1,4652.0,1,0,2240,1,0,NaN
1,1,2,0,1270.0,9,2008,0,1,2016,0,...,1,1,1,6377.0,1,0,1778,1,0,NaN
2,1,2,0,1270.0,9,2008,0,1,2016,0,...,1,1,1,5394.0,0,0,1603,1,0,NaN
3,1,2,0,1270.0,9,2008,0,1,2016,0,...,1,1,1,5695.0,0,0,2387,1,0,NaN
4,1,2,0,1270.0,9,2008,0,1,2016,0,...,1,1,1,5325.0,0,0,2072,1,0,NaN


In [107]:
trainset["OpeningType"][trainset.Store.isin(type1)] = trainset["OpeningType"].fillna(1)
trainset["OpeningType"][trainset.Store.isin(type2)] = trainset["OpeningType"].fillna(2)
trainset.info()

/Users/rfdanti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/rfdanti/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050330 entries, 0 to 1050329
Data columns (total 24 columns):
Store                        1050330 non-null int64
StoreType                    1050330 non-null int64
Assortment                   1050330 non-null int64
CompetitionDistance          1050330 non-null float64
CompetitionOpenSinceMonth    1050330 non-null int64
CompetitionOpenSinceYear     1050330 non-null int64
Promo2                       1050330 non-null int64
Promo2SinceWeek              1050330 non-null int64
Promo2SinceYear              1050330 non-null int64
PromoInterval                1050330 non-null int64
CompetitionOpenSinceDate     1050330 non-null object
Promo2SinceDate              1050330 non-null object
Date                         1050330 non-null object
Customers                    1022401 non-null float64
DayOfWeek                    1050330 non-null int64
Open                         1050330 non-null int64
Promo                        1050330 non-null in

In [111]:
trainset["OpeningType"] = trainset["OpeningType"].fillna(0)

In [114]:
trainset["Date"][trainset["Open"]==1][trainset["DayOfWeek"]==7].unique()

array(['2013-09-29', '2013-12-22', '2013-07-21', '2015-04-05',
       '2013-09-08', '2015-03-29', '2015-07-12', '2013-12-15',
       '2013-09-15', '2014-01-05', '2015-03-08', '2015-03-01',
       '2014-01-12', '2015-07-26', '2013-07-07', '2013-09-22',
       '2015-03-22', '2013-12-29', '2015-03-15', '2015-07-19',
       '2013-07-14', '2015-05-17', '2013-10-20', '2013-11-10',
       '2015-05-10', '2013-08-11', '2013-11-17', '2015-05-31',
       '2013-11-03', '2015-06-07', '2013-10-27', '2015-06-14',
       '2015-05-24', '2013-08-25', '2013-08-18', '2015-04-19',
       '2013-12-01', '2015-06-28', '2013-10-06', '2015-07-05',
       '2015-04-12', '2013-12-08', '2013-07-28', '2015-05-03',
       '2015-06-21', '2013-11-24', '2015-04-26', '2013-09-01',
       '2013-10-13', '2013-08-04', '2014-05-18', '2014-09-28',
       '2013-02-03', '2013-04-28', '2014-10-05', '2013-02-10',
       '2014-06-01', '2014-09-14', '2013-04-21', '2014-05-25',
       '2014-09-21', '2014-04-27', '2014-10-26', '2013-

In [121]:
import datetime
def month(date):
    l=list(map(int, date.split('-')))
    month=int(l[1])
    return month
def year(date):
    l=list(map(int, date.split('-')))
    year=int(l[0])
    return year
def dateofmonth(date):
    l=list(map(int, date.split('-')))
    dateofmonth=int(l[2])
    return dateofmonth
dateofmonth(trainset["Date"][0])

20

In [119]:
trainset["Date"][0]

'2014-10-20'

In [124]:
trainset["month"] = np.nan
trainset["year"] = np.nan
trainset["dateofmonth"] = np.nan

In [128]:
trainset.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,...,Sales,SchoolHoliday,StateHoliday,competition_age_days,affectedbycompetition,affectedbypromo2,OpeningType,month,year,dateofmonth
0,1,2,0,1270.0,9,2008,0,1,2016,0,...,4652.0,1,0,2240,1,0,0.0,10.0,2014.0,20.0
1,1,2,0,1270.0,9,2008,0,1,2016,0,...,6377.0,1,0,1778,1,0,0.0,7.0,2013.0,15.0
2,1,2,0,1270.0,9,2008,0,1,2016,0,...,5394.0,0,0,1603,1,0,0.0,1.0,2013.0,21.0
3,1,2,0,1270.0,9,2008,0,1,2016,0,...,5695.0,0,0,2387,1,0,0.0,3.0,2015.0,16.0
4,1,2,0,1270.0,9,2008,0,1,2016,0,...,5325.0,0,0,2072,1,0,0.0,5.0,2014.0,5.0


In [ ]:
29335

In [133]:
trainset['Date'] = pd.to_datetime(trainset['Date'])
trainset['month'] = trainset['Date'].dt.month

In [134]:
trainset['year'] = trainset['Date'].dt.year

In [135]:
trainset['dateofmonth'] = trainset['Date'].dt.day

In [137]:
trainset["month"] = trainset["month"].astype("int")
trainset["year"] = trainset["year"].astype("int")
trainset["dateofmonth"] = trainset["dateofmonth"].astype("int")

In [138]:
#set categorical type
category_columns = ["Store", "StoreType", "Assortment", "DayOfWeek", "PromoInterval", "StateHoliday",
                   "OpeningType", "month", "year", "dateofmonth"]
     
for column in category_columns:
    trainset[column] = trainset[column].astype('category')

In [141]:
trainset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050330 entries, 0 to 1050329
Data columns (total 27 columns):
Store                        1050330 non-null category
StoreType                    1050330 non-null category
Assortment                   1050330 non-null category
CompetitionDistance          1050330 non-null float64
CompetitionOpenSinceMonth    1050330 non-null int64
CompetitionOpenSinceYear     1050330 non-null int64
Promo2                       1050330 non-null int64
Promo2SinceWeek              1050330 non-null int64
Promo2SinceYear              1050330 non-null int64
PromoInterval                1050330 non-null category
CompetitionOpenSinceDate     1050330 non-null object
Promo2SinceDate              1050330 non-null object
Date                         1050330 non-null datetime64[ns]
Customers                    1022401 non-null float64
DayOfWeek                    1050330 non-null category
Open                         1050330 non-null int64
Promo                    

In [142]:
trainset.to_csv("train_merged_v0.7.csv", index=False)